<a href="https://colab.research.google.com/github/hamed-jamali-software/mp32face/blob/main/coqui_XTTS_v2_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b dev https://github.com/camenduru/xtts2-hf
%cd /content/xtts2-hf
!pip install -q gradio==3.50.2 TTS==0.21.1 langid unidic-lite unidic deepspeed
!pip install -q numpy<2.0.0 -U
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/examples/female.wav -O /content/xtts2-hf/examples/female.wav
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/examples/male.wav -O /content/xtts2-hf/examples/male.wav
!wget https://huggingface.co/spaces/coqui/xtts/resolve/main/ffmpeg.zip -O /content/xtts2-hf/ffmpeg.zip


/content
Cloning into 'xtts2-hf'...
remote: Enumerating objects: 518, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 518 (delta 2), reused 3 (delta 1), pack-reused 512 (from 1)
Receiving objects: 100% (518/518), 716.12 KiB | 1.95 MiB/s, done.
Resolving deltas: 100% (311/311), done.
Error downloading object: examples/female.wav (89a4fa9): Smudge error: Error downloading examples/female.wav (89a4fa9a16b6463f852cf9424f72c3d3c87aa83010e89db534c53fcd1ae12c02): [89a4fa9a16b6463f852cf9424f72c3d3c87aa83010e89db534c53fcd1ae12c02] Object does not exist on the server: [404] Object does not exist on the server

Errors logged to /content/xtts2-hf/.git/lfs/logs/20240902T175137.881776329.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: examples/female.wav: smudge filter lfs failed
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'


In [2]:
import sys
import io, os, stat
import subprocess
import random
from zipfile import ZipFile
import uuid
import time
import torch
import torchaudio
import subprocess
import uuid
import time
import torch
import torchaudio
import langid
import re
from zipfile import ZipFile
from TTS.api import TTS
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.generic_utils import get_user_data_dir
from huggingface_hub import HfApi


In [3]:
os.system('python -m unidic download')

# By using XTTS you agree to CPML license https://coqui.ai/cpml
os.environ["COQUI_TOS_AGREED"] = "1"

HF_TOKEN = os.environ.get("HF_TOKEN")
api = HfApi(token=HF_TOKEN)
repo_id = "coqui/xtts"

# Use newer ffmpeg binary for Ubuntu20 to use denoising for microphone input
print("Export newer ffmpeg binary for denoise filter")
ZipFile("ffmpeg.zip").extractall()
print("Make ffmpeg binary executable")
st = os.stat("ffmpeg")
os.chmod("ffmpeg", st.st_mode | stat.S_IEXEC)

# This will trigger downloading model
print("Downloading if not downloaded Coqui XTTS V2")
from TTS.utils.manage import ModelManager

model_name = "tts_models/multilingual/multi-dataset/xtts_v2"
ModelManager().download_model(model_name)
model_path = os.path.join(get_user_data_dir("tts"), model_name.replace("/", "--"))
print("XTTS downloaded")

Export newer ffmpeg binary for denoise filter
Make ffmpeg binary executable
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2
 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
XTTS downloaded


In [4]:
config = XttsConfig()
config.load_json(os.path.join(model_path, "config.json"))

model = Xtts.init_from_config(config)
model.load_checkpoint(
    config,
    checkpoint_path=os.path.join(model_path, "model.pth"),
    vocab_path=os.path.join(model_path, "vocab.json"),
    eval=True,
    use_deepspeed=True,
)
model.cuda()

DEVICE_ASSERT_DETECTED = 0
DEVICE_ASSERT_PROMPT = None
DEVICE_ASSERT_LANG = None

supported_languages = config.languages


/usr/local/lib/python3.10/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


[2024-09-02 17:55:37,643] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-09-02 17:55:39,794] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.15.0, git-hash=unknown, git-branch=unknown
[2024-09-02 17:55:39,796] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter replace_method is deprecated. This parameter is no longer needed, please remove from your call to DeepSpeed-inference
[2024-09-02 17:55:39,799] [WARNING] [config_utils.py:70:_process_deprecated_field] Config parameter mp_size is deprecated use tensor_parallel.tp_size instead
[2024-09-02 17:55:39,801] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu121/transformer_inference...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/transformer_inference/build.ninja...
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


Time to load transformer_inference op: 82.68424773216248 seconds
[2024-09-02 17:57:03,228] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layers': -1, 'dtype': torch.float32, 'pre_layer_norm': True, 'norm_type': <NormType.LayerNorm: 1>, 'local_rank': -1, 'stochastic_mode': False, 'epsilon': 1e-05, 'mp_size': 1, 'scale_attention': True, 'triangular_masking': True, 'local_attention': False, 'window_size': 1, 'rotary_dim': -1, 'rotate_half': False, 'rotate_every_two': True, 'return_tuple': True, 'mlp_after_attn': True, 'mlp_act_func_type': <ActivationFuncType.GELU: 1>, 'specialized_mode': False, 'training_mp_size': 1, 'bigscience_bloom': False, 'max_out_tokens': 1024, 'min_out_tokens': 1, 'scale_attn_by_inverse_layer_idx': False, 'enable_qkv_quantization': False, 'use_mup': False, 'return_single_tuple': False, 'set_empty_params': False, 'transposed_mode': False, 'use_triton': Fa

Loading extension module transformer_inference...


In [25]:
import textwrap
import subprocess
import uuid
import time
import re
import datetime
import csv
from io import StringIO
from google.colab import files

In [6]:
def split_text(text, max_length=133):
    return textwrap.wrap(text, width=max_length, break_long_words=False)


In [13]:
import numpy as np

def create_silence(duration_seconds, file_path, sample_rate=24000):
    # تعداد نمونه‌ها به ازای مدت زمان
    num_samples = int(duration_seconds * sample_rate)
    # ایجاد آرایه‌ای از صفرها (صداهای خالی)
    silence = np.zeros(num_samples, dtype=np.float32)
    # ذخیره فایل WAV
    torchaudio.save(file_path, torch.tensor([silence]), sample_rate)

In [23]:
def text_to_speech(prompt, language, audio_file_pth=None, mic_file_path=None, use_mic=False, voice_cleanup=False, no_lang_auto_detect=False, agree=True, output_file=None):
    if agree:


        speaker_wav = audio_file_pth




        try:
            prompt = re.sub("([^\x00-\x7F]|\w)(\.|\。|\?)", r"\1 \2\2", prompt)
            print("Generating new audio...")
            # t0 = time.time() # زمان شروع فرآیند تولید صوت
            out = model.inference(
                prompt,
                language,
                *model.get_conditioning_latents(audio_path=speaker_wav, gpt_cond_len=30, gpt_cond_chunk_len=4, max_ref_length=60),
                repetition_penalty=5.0,
                temperature=0.75,
            )
            # inference_time = time.time() - t0
            # print(f"Time to generate audio: {round(inference_time * 1000)} milliseconds")
            # real_time_factor = (time.time() - t0) / out['wav'].shape[-1] * 24000
            # print(f"Real-time factor (RTF): {real_time_factor}")
            torchaudio.save(output_file, torch.tensor(out["wav"]).unsqueeze(0), 24000)
            print("Audio saved to  "+output_file + "  sentence : " + prompt)

        except RuntimeError as e:
            if "device-side assert" in str(e):
                print(f"Exit due to: Unrecoverable exception caused by language:{language} prompt:{prompt}")
                print("Unhandled Exception encounter, please retry in a minute")
                print("Cuda device-assert Runtime encountered need restart")
                if not DEVICE_ASSERT_DETECTED:
                    DEVICE_ASSERT_DETECTED = 1
                    DEVICE_ASSERT_PROMPT = prompt
                    DEVICE_ASSERT_LANG = language

                error_time = datetime.datetime.now().strftime("%d-%m-%Y-%H:%M:%S")
                error_data = [
                    error_time,
                    prompt,
                    language,
                    audio_file_pth,
                    mic_file_path,
                    use_mic,
                    voice_cleanup,
                    no_lang_auto_detect,
                    agree,
                ]
                error_data = [str(e) if type(e) != str else e for e in error_data]
                print(error_data)
                write_io = StringIO()
                csv.writer(write_io).writerows([error_data])
                csv_upload = write_io.getvalue().encode()

                filename = error_time + "_" + str(uuid.uuid4()) + ".csv"
                print("Writing error csv")
                error_api = HfApi()
                error_api.upload_file(
                    path_or_fileobj=csv_upload,
                    path_in_repo=filename,
                    repo_id="coqui/xtts-flagged-dataset",
                    repo_type="dataset",
                )

                print("Writing error reference audio")
                speaker_filename = error_time + "_reference_" + str(uuid.uuid4()) + ".wav"
                error_api.upload_file(
                    path_or_fileobj=speaker_wav,
                    path_in_repo=speaker_filename,
                    repo_id="coqui/xtts-flagged-dataset",
                    repo_type="dataset",
                )

                space = api.get_space_runtime(repo_id=repo_id)
                if space.stage != "BUILDING":
                    api.restart_space(repo_id=repo_id)
                else:
                    print("TRIED TO RESTART but space is building")
            else:
                if "Failed to decode" in str(e):
                    print("Speaker encoding error", str(e))
                else:
                    print("RuntimeError: non device-side assert error:", str(e))
                return None

        return output_file


In [8]:
def generate_speech(prompt, language_code, chunk_id):
    output_file = f"chunk_{chunk_id}.wav"
    language_code = "en"
    audio_file_path = '/content/5.mp3'  # Path to reference audio file if needed
    mic_file_path = None  # Path to microphone recorded audio if needed
    use_microphone = False
    voice_cleanup = True
    disable_lang_auto_detect = True


    speaker_wav = audio_file_path

    # Filtering for microphone input
    lowpassfilter = denoise = trim = loudness = True
    lowpass_highpass = "lowpass=8000,highpass=75," if lowpassfilter else ""
    trim_silence = "areverse,silenceremove=start_periods=1:start_silence=0:start_threshold=0.02,areverse,silenceremove=start_periods=1:start_silence=0:start_threshold=0.02," if trim else ""

    out_filename = speaker_wav + "filter.wav"
    shell_command = f"./ffmpeg -y -i {speaker_wav} -af {lowpass_highpass}{trim_silence} {out_filename}".split(" ")
    subprocess.run(shell_command, capture_output=False, text=True, check=True)
    speaker_wav = out_filename

    return text_to_speech(
        prompt=prompt,
        language=language_code,
        audio_file_pth=speaker_wav,
        mic_file_path=mic_file_path,
        use_mic=use_microphone,
        voice_cleanup=voice_cleanup,
        no_lang_auto_detect=disable_lang_auto_detect,
        agree=True,
        output_file= output_file)

In [9]:
# Function to combine multiple audio files into one
def combine_audio_files(audio_files, output_file):

    with open("file_list.txt", "w") as file_list:
      for audio_file in audio_files:
          file_list.write(f"file '{audio_file}'\n")

    command = ["ffmpeg", "-f", "concat", "-safe", "0", "-i", "file_list.txt", "-c", "copy", output_file]

    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(f"Audio files successfully combined into {output_file}")
        print(result.stdout)
        print(result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during combining audio files: {e}")
        print(e.stdout)
        print(e.stderr)

    for audio_file in audio_files:
      os.remove(audio_file)

    # with open(output_file, 'wb') as outfile:
    #     for audio_file in audio_files:
    #         with open(audio_file, 'rb') as infile:
    #             outfile.write(infile.read())
            # Clean up temporary files
            # os.remove(audio_file)

In [26]:
def process_text_to_speech(text, language_code):
    # Split text into chunks of 200 characters
    chunks = split_text(text)

    sentences = re.split(r'(?<=[.!?]|\[|\])\s+', text)

    all_audio_files = []


    if os.path.isfile("combined_output.wav"):
      os.remove("combined_output.wav")

    total_sentences = len(sentences)
    for idx, sentence in enumerate(sentences):
        print(f"Processing sentence {idx + 1} of {total_sentences}...")
        temp_file = generate_speech(sentence, language_code, idx + 1)
        if os.path.exists(temp_file):
            all_audio_files.append(temp_file)

            # ایجاد فایل صوتی خالی با مدت زمان رندوم
            silence_duration = random.uniform(1, 2)  # مدت زمان رندوم بین 1 تا 2 ثانیه
            silence_file = f"silence_{idx + 1}.wav"
            create_silence(silence_duration, silence_file)
            all_audio_files.append(silence_file)
        else:
            print(f"Failed to generate audio for sentence {idx + 1}.")

    # Combine all audio files into one
    if all_audio_files:
        combined_audio = "combined_output.wav"
        combine_audio_files(all_audio_files, combined_audio)
        print(f"All audio files combined into {combined_audio}")

        files.download("combined_output.wav")

    else:
        print("No audio files to combine.")

In [24]:

# Example usage
prompt_text = """

Hello and welcome to hamjam.

I’m hamjam, and in today’s video, we’re going to explore [how to get motivated, stay disciplined, be productive and ultimately re-invent yourself on your self-improvement journey].

If you’re passionate about living a well-ordered life and achieving financial success, you’re in the right place! Be sure to hit that like button if you enjoy the content, and don’t forget to subscribe to stay updated with all our latest videos.

Let’s dive in and start transforming your life together!


The greatest achievements often come right after our worst defeats. It's time to get back to work and stop listening to your own excuses. Get off the podcasts, step away from social media, and silence the noise around you. You need to return to your mental lab. This work is not just an apology to yourself; it’s an apology for wasting time, for holding yourself back, and for sabotaging your own success.

If you're reading this, it's clear that you're sick and tired of being sick and tired. You've had enough of being average. There comes a point when being mediocre becomes nauseating, when it makes you want to throw up. If you don’t feel this way, maybe this message isn’t for you. But if you do, it’s time for version 2.0 of yourself to emerge—the version that is committed to your goals no matter what, that feels the fear but does it anyway, and that attacks your goals relentlessly without procrastination.

I’m in this phase too. For too long, I was stuck in a rut, putting off achieving the goals I set for myself. Instead of doing the work that would move my life forward, I indulged in activities that were no good for me—this is called escapism. I avoided chasing my goals because I was afraid of being laughed at. I talked a lot about achieving big things, but I never took action. I couldn't focus on any task because I was constantly distracted. As soon as I felt any resistance, I would seek out cheap dopamine. But eventually, something snapped. I realized that enough is enough. How much longer are we going to spend debating our goals before we actually get up and do something? Ask yourself: what would be worse—the pain of getting to the end of your life and realizing you lived in regret, or the pain of enduring a hard task that would give you a sense of accomplishment? You know the answer.

So let me tell you how we’re going to make the greatest comeback of our lives and evolve into the best version of ourselves. It's time for 2.0. You’ve got to disappear for a few months and focus on self-improvement. I don’t mean in some emo, antisocial way, but in a focused, goal-oriented manner. What you have now is not a dream because you will make it a reality.

Let’s quickly summarize using an example like Sasuke. He spent his entire youth training and seeking power to become stronger and avenge his clan. He had a single-minded focus on his goal. We need to apply this same mindset to our own lives. Attack your goals with unwavering confidence. Don’t waste time sitting around debating your goals; you need to build momentum, and success will follow. This momentum comes from an attacking mindset.

Over the next few months, you need to improve every aspect of your life. The best place to start is by establishing a powerful routine. It might sound cliché, but once you create a strong routine, you’ll realize just how powerful it can be. When you set an early wake-up time and tackle the hardest task of the day first thing in the morning, it breeds confidence. The saying goes, "If you win the morning, you win the day," and if you keep winning days, you win the weeks, months, and eventually the year. The momentum from the morning carries through the day, and when you consistently do what you say you will do, it breeds confidence.

Personally, I wake up at 6 a.m., start my day with an ice bath and a workout. This sets the tone for the day, giving me the confidence to attack the rest of my goals. For too long, I unknowingly lived resenting myself for being unproductive and lazy, but that’s no longer the case. Just following a morning routine like Huberman's is the best choice. Start your day with something challenging, and build from there. If you win the morning, the rest of the day falls into place.

During this comeback, you must be disciplined. You have to consistently follow through on your tasks and build a ferocious mindset. Believe in yourself and your abilities. Over time, this discipline will turn into a habit, allowing you to live with greater confidence.

To sum it up, you have to improve your physique by going to the gym consistently, upgrade your knowledge by reading books and listening to informative podcasts, and most importantly, evolve your mindset. Why walk through life thinking you're anything less than capable? The alternative is simply not acceptable. Look at the best athletes and successful people you admire; they have built a mindset of ferocity and belief in themselves, no matter what. You can do the same. Use the pain you’ve experienced as fuel to succeed, and remember: this is your comeback.


I hope you found today’s video helpful and inspiring. If you enjoyed it, please give us a thumbs up and share it with anyone who might benefit from it.

Remember to subscribe if you haven’t already, so you never miss out on our latest tips and insights. Feel free to leave a comment below with your thoughts or any questions you have — I love hearing from you!


"""
process_text_to_speech(prompt_text, language_code="en")

Processing sentence 1 of 66...
Generating new audio...
Audio saved to  chunk_1.wav  sentence : 

Hello and welcome to hamjam ..
Processing sentence 2 of 66...
Generating new audio...
Audio saved to  chunk_2.wav  sentence : I’m hamjam, and in today’s video, we’re going to explore [how to get motivated, stay disciplined, be productive and ultimately re-invent yourself on your self-improvement journey].
Processing sentence 3 of 66...
Generating new audio...
Audio saved to  chunk_3.wav  sentence : If you’re passionate about living a well-ordered life and achieving financial success, you’re in the right place!
Processing sentence 4 of 66...
Generating new audio...
Audio saved to  chunk_4.wav  sentence : Be sure to hit that like button if you enjoy the content, and don’t forget to subscribe to stay updated with all our latest videos ..
Processing sentence 5 of 66...
Generating new audio...
Audio saved to  chunk_5.wav  sentence : Let’s dive in and start transforming your life together!
Proces

In [ ]:
command = [
    "ffmpeg",
    "-y",
    "-i", "combined_output.wav",
    "-filter:a", "atempo=0.93",
    "output_final.wav"
]


# اجرای فرمان FFmpeg
try:
    result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print("Command executed successfully.")




    print(result.stdout)
except subprocess.CalledProcessError as e:
    print("Error occurred:")
    print(e.stderr)

Command executed successfully.

